# Vitamix Forecasting Models

# Global
    * Data Load and Preparation

In [22]:
# Packages
import snowflake.connector
import pandas as pd
import os
import numpy as np

from datetime import datetime, timedelta

from tqdm import tqdm

# import the regression module from pycaret   
import pycaret.regression as pycr

# import the time seris module from pycaret

import pycaret.time_series as pycts

# copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [2]:
# Query Snowflake

def snowflake_to_pandas(connection_params, query):
    try:
        # Establish a connection to Snowflake
        conn = snowflake.connector.connect(**connection_params)

        # Execute the SQL query and fetch the results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# SQL command 
query = 'SELECT * FROM "ANALYTICS"."FORECASTING"."sales_fcast_combined_v"'

# Call the function to retrieve the data into a Pandas DataFrame
result_df = snowflake_to_pandas(connection_params, query)

if result_df is not None:
    print(result_df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to retrieve data from Snowflake.")

   DEP_ENT       MONTH     NET_SALES  BUDGET_AMOUNT  FORECAST
0  210_310  2020-09-01  7.449778e+05      1330384.0       NaN
1  210_310  2020-10-01  5.257454e+05       936992.0       NaN
2  210_310  2020-11-01  1.998459e+06      1681287.0       NaN
3  210_310  2020-12-01  4.443028e+05       734573.0       NaN
4  210_310  2021-01-01  3.309476e+05            NaN       NaN


In [3]:
### Data prep ###

result_df["MONTH"] = pd.to_datetime(result_df["MONTH"]) # convert month field to date
df_all = result_df[result_df['MONTH'] <= pd.Timestamp(2023,6,1)] # we have data through July '23 when we are training these models. 
#Some random months will have data that we want to remove (* Want to test without July though)
df_all = df_all[["DEP_ENT", "MONTH", "NET_SALES"]] # select fields of interest
df_all = df_all.sort_values(['DEP_ENT', 'MONTH']) # reorder dataframe

In [4]:
### Spot checks ###

# check the unique time_series. 14 different department-entity combinations
print(df_all['DEP_ENT'].nunique())
print("-----")

# check how many months for each dep-ent. 3 dep-ent do not have all months of data
print(df_all.groupby(['DEP_ENT']).size())
print("-----")

# check for nulls. No nulls
print(df_all.isnull().sum())

14
-----
DEP_ENT
160_155    90
170_155    90
200_155    90
200_310    90
210_155    90
210_165    90
210_310    90
220_155    90
220_310    84
240_155    90
250_155    90
250_165    56
250_310    90
260_155    53
dtype: int64
-----
DEP_ENT      0
MONTH        0
NET_SALES    0
dtype: int64


# Finished with global data work above
__________________________________________________________________________________________________________________________________________________________________________________________________

### 1
# Time Series Forecasting with PyCaret Regression

Pycaret 3.04 regression documentation: https://pycaret.readthedocs.io/en/stable/api/regression.html

In [5]:
### Data Prep for Regression ###

df_all_rg = df_all

# extract month and year from dates
df_all_rg['Month'] = [i.month for i in df_all_rg['MONTH']]
df_all_rg['Year'] = [i.year for i in df_all_rg['MONTH']]

# create a sequence of numbers
df_all_rg['Series'] = np.arange(1,len(df_all_rg)+1)

# Notes on below TS loop
* Manual changes:
  * periods to forecast ahead using the next_dates variable 

In [ ]:
### Regression Loop ###

# Create empty dataframes
all_results_rg = pd.DataFrame()
all_predictions_rg = pd.DataFrame()

# list of each dep-ent
all_ts_rg = df_all_rg['DEP_ENT'].unique()

for i in tqdm(all_ts_rg):
    
    # temp df for a dep-ent
    df_subset = df_all_rg[df_all_rg['DEP_ENT'] == i]
    
    # trim
    df_subset_trim = df_subset[['Series', 'Year', 'Month', 'NET_SALES']] 
    
    # initialize setup
    s = pycr.setup(data = df_subset_trim, target = 'NET_SALES', session_id = 123)
    
    # model training and selection
    best_model = pycr.compare_models()
    
    # capture the compare result grid and store best model in list
    p = pycr.pull().iloc[0:1]
    p['DEP_ENT'] = str(i)
    
    all_results_rg = all_results_rg.append(p)
    
    ### New data to predict ###

    # max date from original dataset
    max_timestamp = df_subset['MONTH'].max()

    # create dataframe for future dates, in this case the rest of 2023
    next_dates = [max_timestamp.replace(day=1) + pd.DateOffset(months=i) for i in range(1, 7)] # Need to change range based on how many periods ahead you want to predict
    new_dates_df = pd.DataFrame({'MONTH': next_dates})
    new_dates_df["MONTH"] = pd.to_datetime(new_dates_df["MONTH"]) 

    # extract month and year from dates
    new_dates_df['Month'] = [i.month for i in new_dates_df['MONTH']]
    new_dates_df['Year'] = [i.year for i in new_dates_df['MONTH']]

    # create a sequence of numbers
    new_dates_df['Series'] = np.arange(1,len(new_dates_df)+1)

    # select cols
    new_dates_df = new_dates_df[['Series', 'Year', 'Month']] 
    
    # predict on new data
    predictions = pycr.predict_model(best_model, data = new_dates_df)
    
    ### Combine predictions in to all_predictions dataframe ###
    
    pred_temp = predictions.reset_index()
    
    pred_comb = pd.concat([pd.DataFrame({'MONTH': next_dates}), pred_temp['prediction_label']], axis = 1)
    pred_comb['DEP_ENT'] = i # add dep
    pred_comb.rename(columns = {'prediction_label':'PRED'}, inplace = True)
    
    # append to master dataframe
    all_predictions_rg = all_predictions_rg.append(pred_comb)

__________________________________________________________________________________________________________________________________________________________________________________________________

### 2
# Time Series Forecasting with PyCaret Time Series

Pycaret 3.04 time series documentation: https://pycaret.readthedocs.io/en/stable/api/time_series.html

In [24]:
### Data Prep for Time Series Anlysis ###

df_all_ts = df_all

# create time series dataset
df_all_ts = df_all_ts[["DEP_ENT", "MONTH", "NET_SALES"]]

# Notes on below TS loop
* Manual changes:
  * periods to include in the dates dataset created using next_dates
  * forecast horizon (fh) in predict_model function

In [43]:
### Time Series Loop ###

# Create empty dataframes
all_results_ts = pd.DataFrame()
all_predictions_ts = pd.DataFrame()

# list of each dep-ent
all_ts_ts = df_all_ts['DEP_ENT'].unique()

for i in tqdm(all_ts_ts):
    
    # temp df for a dep-ent
    df_subset = df_all_ts[df_all_ts['DEP_ENT'] == i]
    
    # trim
    df_subset_trim = df_subset[['MONTH', 'NET_SALES']] 
    
    # create series
    df_subset_s = df_subset_trim.set_index('MONTH')['NET_SALES']
    
    # set frequency of series
    df_subset_s = df_subset_s.asfreq('MS') 

    # test and train
    ts_train = round(len(df_subset_trim.index) * .8) # 80% train
    ts_test = len(df_subset_trim.index) - round(len(df_subset_trim.index) * .8) # 20% test
    
    # initialize setup
    s = pycts.setup(df_subset_s, numeric_imputation_target = "mean", fh = ts_test, session_id = 123)

    # model training and selection
    # - exclude = ["naive", "grand_means"]
    best_model = pycts.compare_models()
    
    # capture the compare result grid and store best model in list
    p = pycts.pull().iloc[0:1]
    p['DEP_ENT'] = str(i)
    
    all_results_ts = all_results_ts.append(p)
    
    ### Predict future periods ###
    
    # max date from original dataset
    max_timestamp = df_subset_trim['MONTH'].max()
    
    # create dataframe for future dates, in this case the rest of 2023. This is not for predicting but to join back up with predictions
    next_dates = [max_timestamp.replace(day=1) + pd.DateOffset(months=i) for i in range(1, 7)] # Need to change range based on how many periods ahead you want to predict
    new_dates_df = pd.DataFrame({'MONTH': next_dates})
    new_dates_df["MONTH"] = pd.to_datetime(new_dates_df["MONTH"]) 
    
    # predict in unseen future
    predictions = pycts.predict_model(best_model, fh=6) # Need to change range fh for how many periods ahead you want to predict
    
    pred_temp = predictions.reset_index()
    
    pred_comb = pd.concat([pd.DataFrame({'MONTH': next_dates}), pred_temp['y_pred']], axis = 1)
    pred_comb['DEP_ENT'] = i # add dep
    pred_comb.rename(columns = {'y_pred':'PRED'}, inplace = True)
    
    # append to master dataframe
    all_predictions_ts = all_predictions_ts.append(pred_comb)

  0%|          | 0/14 [00:00<?, ?it/s]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(90, 1)"
5,Transformed data shape,"(90, 1)"
6,Transformed train set shape,"(72, 1)"
7,Transformed test set shape,"(18, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
grand_means,Grand Means Forecaster,0.7977,0.8547,1053265.9363,1403731.8462,0.7008,0.4233,-0.5374,0.0267
arima,ARIMA,0.8037,0.8586,1061451.1860,1410308.2641,0.7087,0.4260,-0.5640,0.0333
croston,Croston,0.8728,0.9250,1151466.3385,1519451.3422,0.7828,0.4540,-0.9183,0.0200
auto_arima,Auto ARIMA,1.0115,0.9801,1354243.2415,1611787.8779,0.8862,0.4952,-0.9171,0.1000
theta,Theta Forecaster,1.1800,1.1134,1571089.5330,1832350.3154,1.1042,0.5611,-2.1640,0.0233
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.3122,1.1909,1760022.2466,1963314.9082,1.2125,0.5963,-2.4527,0.1467
polytrend,Polynomial Trend Forecaster,1.3209,1.2308,1769582.2458,2027109.1468,1.1868,0.6019,-2.5287,0.0200
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.3221,1.2317,1771161.5497,2028565.4072,1.1879,0.6023,-2.5371,0.0900
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.3263,1.2340,1777295.2050,2032650.4381,1.1956,0.6026,-2.5751,0.1000
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.3270,1.2347,1778429.6522,2033861.9546,1.1974,0.6026,-2.5881,0.0900


  7%|▋         | 1/14 [00:12<02:39, 12.24s/it]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(90, 1)"
5,Transformed data shape,"(90, 1)"
6,Transformed train set shape,"(72, 1)"
7,Transformed test set shape,"(18, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


...

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.1520,1.2578,217230.5206,280220.8537,27.4182,0.5796,0.1917,0.2600
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,1.1663,1.3033,220167.3744,290744.1320,31.2027,0.5532,0.1155,0.3167
snaive,Seasonal Naive Forecaster,1.1985,1.2402,225701.7823,277107.6545,22.3300,0.6050,0.2107,0.0367
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.2718,1.3570,239878.2232,303201.4166,29.5957,0.6273,0.0456,0.3700
grand_means,Grand Means Forecaster,1.2868,1.5113,244710.6215,341262.4117,31.3057,0.5591,-0.2241,0.0233
arima,ARIMA,1.3121,1.3212,246952.4698,295379.7770,22.4059,0.6668,0.1125,0.0400
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.3237,1.3771,248377.1490,305749.0899,26.5473,0.6498,0.0338,0.1833
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.3337,1.3994,250614.9861,310898.0155,29.5308,0.6340,-0.0108,0.3433
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.3718,1.4318,257707.9642,318600.0111,36.3570,0.6389,-0.0639,0.1733
stlf,STLF,1.3756,1.3508,260639.7985,304162.2741,27.2601,0.6941,0.0046,0.0300


 93%|█████████▎| 13/14 [03:34<00:16, 16.79s/it]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(53, 1)"
5,Transformed data shape,"(53, 1)"
6,Transformed train set shape,"(42, 1)"
7,Transformed test set shape,"(11, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,1.3929,1.5466,2028.6996,2750.1185,0.4794,0.4173,-0.3022,0.0767
arima,ARIMA,1.3930,1.5452,2029.5760,2748.5019,0.4812,0.4176,-0.3076,0.0400
grand_means,Grand Means Forecaster,1.3957,1.5475,2034.4576,2752.8021,0.4805,0.4185,-0.3049,0.0233
naive,Naive Forecaster,1.3968,1.5290,2065.3003,2754.2822,0.5376,0.4235,-0.4375,0.0333
exp_smooth,Exponential Smoothing,1.4467,1.5711,2099.0318,2785.0159,0.4583,0.4389,-0.3085,0.0267
theta,Theta Forecaster,1.4480,1.5847,2112.0139,2818.6240,0.4825,0.4358,-0.3289,0.0267
croston,Croston,1.4544,1.5587,2131.1461,2784.1650,0.4828,0.4410,-0.2806,0.0167
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.5908,1.6825,2409.8932,3104.5106,0.6196,0.4820,-0.7383,0.0767
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,1.6036,1.6307,2449.5356,3028.6899,0.6169,0.4890,-0.6282,0.1600
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.6348,1.7064,2490.1490,3174.2738,0.6397,0.4946,-0.8044,0.0733


100%|██████████| 14/14 [03:45<00:00, 16.13s/it]
